In [3]:
from __future__ import division
from __future__ import print_function

import time
import argparse
import sys
print(sys.version)
print(sys.path)
import numpy as np

import torch
import torch.nn.functional as F
import torch.optim as optim
from torchvision.utils import make_grid, save_image

from tensorboardX import SummaryWriter
from tqdm import tqdm
from copy import deepcopy


from utils import *
from models import *
from fid_score import *
from inception_score import *

from real_imgs import *



3.9.5 (v3.9.5:0a7dcbdb13, May  3 2021, 13:17:02) 
[Clang 6.0 (clang-600.0.57)]
['/Users/ryomacmini/work/codes/MNIST_Calculator', '/Users/ryomacmini/.vscode/extensions/ms-toolsai.jupyter-2021.8.2041215044/pythonFiles', '/Users/ryomacmini/.vscode/extensions/ms-toolsai.jupyter-2021.8.2041215044/pythonFiles/lib/python', '/Library/Frameworks/Python.framework/Versions/3.9/lib/python39.zip', '/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9', '/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/lib-dynload', '', '/Users/ryomacmini/work/codes/TransGAN/.venv/lib/python3.9/site-packages', '/Users/ryomacmini/work/codes/TransGAN/.venv/lib/python3.9/site-packages/IPython/extensions', '/Users/ryomacmini/.ipython']


In [ ]:
label_list = ['bird','frog']

In [9]:

if __name__ == '__main__':
    r = RealImage()  # ここで一度だけダウンロードと transform が行われる。
    real_images = r.real_imgs_row_data(label_list)
    print(real_images.size())
    print('complete create cifar_image_set')

Files already downloaded and verified
torch.Size([10000, 3, 32, 32])


In [3]:
img_size = 32
transform = transforms.Compose([transforms.Resize(size=(img_size, img_size)),transforms.RandomHorizontalFlip(),transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=30, shuffle=True)

Files already downloaded and verified


In [24]:
train_set[1]

(tensor([[[-0.3804, -0.3176, -0.2863,  ..., -0.1765, -0.0118,  0.2078],
          [-0.4431, -0.3961, -0.2471,  ..., -0.0196,  0.1373,  0.0980],
          [-0.4745, -0.4667, -0.3804,  ..., -0.0980,  0.0902,  0.0980],
          ...,
          [-0.2706, -0.5216, -0.6706,  ...,  0.2078,  0.2235,  0.3725],
          [ 0.0275, -0.0353, -0.1922,  ...,  0.2471,  0.2235,  0.2941],
          [ 0.1216,  0.1216,  0.1216,  ...,  0.2784,  0.2392,  0.2784]],
 
         [[-0.3647, -0.2941, -0.2549,  ..., -0.1843,  0.0745,  0.3882],
          [-0.4275, -0.3725, -0.2235,  ..., -0.0196,  0.2000,  0.2549],
          [-0.4588, -0.4510, -0.3569,  ..., -0.0980,  0.1451,  0.2157],
          ...,
          [-0.3490, -0.5843, -0.7333,  ...,  0.2549,  0.2078,  0.3098],
          [-0.0510, -0.1059, -0.2706,  ...,  0.2627,  0.1922,  0.2078],
          [ 0.0431,  0.0510,  0.0431,  ...,  0.2235,  0.1608,  0.1608]],
 
         [[-0.4510, -0.4431, -0.4431,  ..., -0.2549,  0.0667,  0.4667],
          [-0.5216, -0.5137,

In [23]:
len(train_set[1])
# train_set[0][0].shape

2